## Produce daily Origin-Destination trip matrix for NYC Resident borough to all U.S. counties using csv generated by EDM 

### Separates count for trips where origin == destination (i.e. "Home visit"), and all other visits

In [1]:
import pandas as pd
import numpy as np
import os
import time

In [2]:
from geo import stco,sub

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
df = pd.read_csv('../data/daily_nycto_us_county_outflow 201009.csv')

In [5]:
df['not_home_visits'] = df['visits'] - df['at_home_visits']

In [6]:
dff = pd.pivot_table(df,values=['not_home_visits'],index=['date_start','origin_fips'],columns=['destination_fips'],aggfunc=np.sum,fill_value=0)
dff.columns = dff.columns.droplevel()
dff.head()

destination_fips        1001   1003   1005   1007   1009   1011   1013   \
date_start origin_fips                                                    
2019-01-01 36005            0      0      0      0      0      0      0   
           36047            0      0      0      0      0      0      1   
           36061            0      5      0      0      0      0      0   
           36081            1      0      0      0      0      0      0   
           36085            0      0      0      0      0      0      0   

destination_fips        1015   1017   1019   1021   1023   1025   1027   \
date_start origin_fips                                                    
2019-01-01 36005            1      0      0      0      0      0      0   
           36047            1      0      0      0      0      0      0   
           36061            1      0      0      0      0      0      1   
           36081            0      0      0      0      0      0      0   
           36085            0      0      0      0      0      0      0   

destination_fips        1029   1031   1033   1035   1037   1039   1041   \
date_start origin_fips                                                    
2019-01-01 36005            0      1      0      0      0      0      0   
           36047            0      0      0      1      0      0      0   
           36061            0      0      2      0      0      0      0   
           36081            0      0      0      0      0      0      0   
           36085            0      0      0      0      0      0      0   

destination_fips        1043   1045   1047   1049   1051   1053   1055   \
date_start origin_fips                                                    
2019-01-01 36005            0      0      0      0      0      0      0   
           36047            1      0      0      0      2      0      0   
           36061            3      0      0      0      0      1      1   
           36081            1      0      0      0      0      0      0   
           36085            0      0      0      0      0      0      0   

destination_fips        1057   1059   1061   1063   1065   1067   1069   \
date_start origin_fips                                                    
2019-01-01 36005            0      2      0      0      0      0      0   
           36047            0      0      0      0      0      0      0   
           36061            0      0      0      0      0      0      1   
           36081            0      0      0      0      0      0      0   
           36085            0      0      0      0      0      0      0   

destination_fips        1071   1073   1075   1077   1079   1081   1083   \
date_start origin_fips                                                    
2019-01-01 36005            0      1      0      0      0      3      1   
           36047            0      5      0      0      0      1      0   
           36061            0     24      0      0      0      3      0   
           36081            0      3      0      0      0      1      0   
           36085            0      0      0      0      0      0      0   

destination_fips        1085   1087   1089   1091   1093   1095   1097   \
date_start origin_fips                                                    
2019-01-01 36005            0      0      0      0      0      0      1   
           36047            0      1      3      0      0      0      4   
           36061            0      0      4      2      0      0     14   
           36081            0      2      7      0      0      0      1   
           36085            0      0      4      0      0      0      0   

destination_fips        1099   1101   1103   1105   1107   1109   1111   \
date_start origin_fips                                                    
2019-01-01 36005            0      0      0      0      0      0      0   
           36047            0      7      0      0      1      0      0   
           36061  

In [7]:
df_home = df[df['at_home_visits'] > 0]

In [8]:
dff_home = pd.pivot_table(df_home,values=['at_home_visits'],index=['date_start','origin_fips'],columns=['destination_fips'],aggfunc=np.sum,fill_value=0,margins=True)
dff_home.columns = dff_home.columns.droplevel()
dff_home = dff_home[['All']].rename(columns={'All':'Home'})
dff_home.head()

destination_fips          Home
date_start origin_fips        
2019-01-01 36005         83990
           36047        128922
           36061         62134
           36081        122920
           36085         32395

In [9]:
df_device = df[['date_start','origin_fips','device_count','completely_home_device_count']]
df_device = df_device.drop_duplicates(ignore_index=True)

In [10]:
dff_device = df_device.set_index(['date_start','origin_fips']).sort_index(ascending=True)
dff_device.head()

device_count  completely_home_device_count
date_start origin_fips                                            
2019-01-01 36005              109524                         55340
           36047              165090                         77727
           36061               86687                         36253
           36081              148363                         70510
           36085               36884                         16432

In [11]:
df_master = pd.concat([dff_device,dff_home,dff],axis=1)

In [12]:
df_master = df_master.reset_index()

In [13]:
df_master.head()

date_start origin_fips  device_count  completely_home_device_count    Home  \
0  2019-01-01       36005      109524.0                       55340.0   83990   
1  2019-01-01       36047      165090.0                       77727.0  128922   
2  2019-01-01       36061       86687.0                       36253.0   62134   
3  2019-01-01       36081      148363.0                       70510.0  122920   
4  2019-01-01       36085       36884.0                       16432.0   32395   

   1001  1003  1005  1007  1009  1011  1013  1015  1017  1019  1021  1023  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   0.0   0.0   0.0   0.0   
2   0.0   5.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   
3   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   1025  1027  1029  1031  1033  1035  1037  1039  1041  1043  1045  1047  \
0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   1.0   0.0   0.0   
2   0.0   1.0   0.0   0.0   2.0   0.0   0.0   0.0   0.0   3.0   0.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   1049  1051  1053  1055  1057  1059  1061  1063  1065  1067  1069  1071  \
0   0.0   0.0   0.0   0.0   0.0   2.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   2.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   0.0   0.0   1.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   1073  1075  1077  1079  1081  1083  1085  1087  1089  1091  1093  1095  \
0   1.0   0.0   0.0   0.0   3.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   5.0   0.0   0.0   0.0   1.0   0.0   0.0   1.0   3.0   0.0   0.0   0.0   
2  24.0   0.0   0.0   0.0   3.0   0.0   0.0   0.0   4.0   2.0   0.0   0.0   
3   3.0   0.0   0.0   0.0   1.0   0.0   0.0   2.0   7.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   4.0   0.0   0.0   0.0   

   1097  1099  1101  1103  1105  1107  1109  1111  1113  1115  1117  1119  \
0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0   
1   4.0   0.0   7.0   0.0   0.0   1.0   0.0   0.0   0.0   2.0   0.0   0.0   
2  14.0   0.0   7.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0   1.0   
3   1.0   0.0   2.0   3.0   0.0   0.0   1.0   0.0   0.0   0.0   1.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   

   1121  1123  1125  1127  1129  1131  1133  2013  2016  2020  2050  2060  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   2.0   0.0   0.0   
2   0.0   0.0   4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   

   2068  2070  2090  2100  2105  2110  2122  2130  2150  2158  2164  2170  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   1.0   0.0   2.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3   0.0   0.0   2.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   2180  2185  2188  2195  2198  2220  2230  2240  2261  2275  2282  2290  \
0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2   0.0   0.0   0.0   0.0   0.0   0.0   0.0 

In [16]:
df_master.to_excel('output/Destinations/dest_coxco_daily_1009.xlsx')